In [1]:
!pip install ultralytics

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.2.90 🚀 Python-3.10.12 torch-2.4.0+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
Setup complete ✅ (12 CPUs, 53.0 GB RAM, 32.8/112.6 GB disk)


In [2]:
!pip install supervision

from IPython import display
display.clear_output()

import supervision as sv
print("supervision.__version__:", sv.__version__)

supervision.__version__: 0.23.0


In [3]:
MODEL = "yolov10x.pt"

In [4]:
from ultralytics import YOLO

model = YOLO(MODEL)
model.fuse()

100%|██████████| 61.4M/61.4M [00:00<00:00, 252MB/s]


YOLOv10x summary (fused): 503 layers, 31,738,160 parameters, 0 gradients, 170.6 GFLOPs


In [5]:
source = '/content/WhatsApp Video 2024-09-06 at 10.37.44.mp4'

In [6]:
CLASS_NAMES_DICT = model.model.names
selected_classes = [2, 3, 5, 7]

In [7]:
import supervision as sv
import numpy as np

In [8]:
LINE_START = sv.Point(50, 1500)
LINE_END = sv.Point(3840-50, 1500)

In [9]:
target="/content/result.mp4"

In [10]:
# # create BYTETracker instance
# # byte_tracker = sv.ByteTrack(track_thresh=0.25, track_buffer=30, match_thresh=0.8, frame_rate=30)

# byte_tracker = sv.ByteTrack(frame_rate=30)


# # create VideoInfo instance
# video_info = sv.VideoInfo.from_video_path(source)

# # create frame generator
# generator = sv.get_video_frames_generator(source)

# # create LineZone instance, it is previously called LineCounter class
# line_zone = sv.LineZone(start=LINE_START, end=LINE_END)

# # create instance of BoxAnnotator
# # box_annotator = sv.BoxAnnotator(thickness=4, text_thickness=4, text_scale=2)
# box_annotator = sv.BoxAnnotator(thickness=2)
# label_annotator = sv.LabelAnnotator(text_position=sv.Position.CENTER, text_scale=0.2, text_thickness=1)


# # create LineZoneAnnotator instance, it is previously called LineCounterAnnotator class
# line_zone_annotator = sv.LineZoneAnnotator(thickness=4, text_thickness=4, text_scale=2)

# # define call back function to be used in video processing
# def callback(frame: np.ndarray, index:int) -> np.ndarray:
#     # model prediction on single frame and conversion to supervision Detections
#     results = model(frame)[0]
#     detections = sv.Detections.from_ultralytics(results)
#     # only consider class id from selected_classes define above
#     detections = detections[np.isin(detections.class_id, selected_classes)]
#     # tracking detections
#     detections = byte_tracker.update_with_detections(detections)
#     labels = [
#         f"#{tracker_id} {model.model.names[class_id]} {confidence:0.2f}"
#         for confidence, class_id, tracker_id
#         in zip(detections.confidence, detections.class_id, detections.tracker_id)
#     ]
#     # annotated_frame = trace_annotator.annotate(
#     #     scene=frame.copy(),
#     #     detections=detections

#     annotated_frame=box_annotator.annotate(
#         scene=annotated_frame,
#         detections=detections)
#     annotated_frame = label_annotator.annotate(
#     scene=annotated_frame,  # use the frame already annotated with bounding boxes
#     detections=detections,
#     labels=labels
# )

#     # update line counter
#     line_zone.trigger(detections)
#     # return frame with box and line annotated result
#     return  line_zone_annotator.annotate(annotated_frame, line_counter=line_zone)

# # process the whole video
# sv.process_video(
#     source_path = source,
#     target_path = target,
#     callback=callback
# )


In [11]:
# create BYTETracker instance
byte_tracker = sv.ByteTrack(frame_rate=30)

# create VideoInfo instance
video_info = sv.VideoInfo.from_video_path(source)

# create frame generator
generator = sv.get_video_frames_generator(source)

# create LineZone instance, previously called LineCounter class
line_zone = sv.LineZone(start=LINE_START, end=LINE_END)

# create BoxAnnotator instance
box_annotator = sv.BoxAnnotator(thickness=2)
label_annotator = sv.LabelAnnotator(text_position=sv.Position.CENTER, text_scale=0.2, text_thickness=1)

# create LineZoneAnnotator instance, previously called LineCounterAnnotator class
line_zone_annotator = sv.LineZoneAnnotator(thickness=4, text_thickness=4, text_scale=2)

# define callback function to be used in video processing
def callback(frame: np.ndarray, index: int) -> np.ndarray:
    # model prediction on single frame and conversion to supervision Detections
    results = model(frame)[0]
    detections = sv.Detections.from_ultralytics(results)

    # only consider class id from selected_classes defined above
    detections = detections[np.isin(detections.class_id, selected_classes)]

    # tracking detections
    detections = byte_tracker.update_with_detections(detections)

    # labels for annotations
    labels = [
        f"#{tracker_id} {model.model.names[class_id]} {confidence:0.2f}"
        for confidence, class_id, tracker_id
        in zip(detections.confidence, detections.class_id, detections.tracker_id)
    ]

    # Annotate frame with boxes and labels
    annotated_frame = box_annotator.annotate(
        scene=frame.copy(),
        detections=detections
    )

    # Check if annotated_frame is not None before using it
    if annotated_frame is not None:
        annotated_frame = label_annotator.annotate(
            scene=annotated_frame,  # use the frame already annotated with bounding boxes
            detections=detections,
            labels=labels
        )

    # update line counter
    line_zone.trigger(detections)

    # return frame with box and line annotated result
    return line_zone_annotator.annotate(annotated_frame, line_counter=line_zone)

# process the whole video
sv.process_video(
    source_path=source,
    target_path=target,
    callback=callback
)


Streaming output truncated to the last 5000 lines.
0: 640x384 3 persons, 6 cars, 2 motorcycles, 1 truck, 2 traffic lights, 17.8ms
Speed: 2.0ms preprocess, 17.8ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 3 persons, 6 cars, 2 motorcycles, 2 traffic lights, 19.0ms
Speed: 2.0ms preprocess, 19.0ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 4 persons, 9 cars, 2 motorcycles, 1 traffic light, 18.3ms
Speed: 2.0ms preprocess, 18.3ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 3 persons, 9 cars, 2 motorcycles, 2 traffic lights, 18.9ms
Speed: 2.0ms preprocess, 18.9ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 4 persons, 10 cars, 2 motorcycles, 3 traffic lights, 18.3ms
Speed: 2.5ms preprocess, 18.3ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 3 persons, 6 cars, 2 motorcycles, 3 traffic lights, 18.4ms
Speed: 2.0ms preprocess, 18.4m